# $P_{LM}(\text{Answer} | \text{Prompt} + \text{Question})$ on Autism Questionnaires

See [questionnaire_max_variance.ipynb](https://github.com/amanb2000/Emo_LLM/blob/main/notebooks/questionnaire_max_variance.ipynb) for details on method. 

## Dataset

Questionnaires are in `datasets/webgazer_AQ_20240613.jsonl` and `datasets/webgazer_SRS_AR_SR_20240613.jsonl` (courtesy of Tessa Rusch and Brenna Outten)

```json
// AQ
{"id": 0, "question": "I prefer to do things with others rather than on my own.", "answers": ["Definitely agree", "Slightly agree", "Slightly disagree", "Definitely disagree"]}
{"id": 1, "question": "I prefer to do things the same way over and over again.", "answers": ["Definitely agree", "Slightly agree", "Slightly disagree", "Definitely disagree"]}
{"id": 2, "question": "If I try to imagine something, I find it very easy to create a picture in my mind.", "answers": ["Definitely agree", "Slightly agree", "Slightly disagree", "Definitely disagree"]}


// SRS
{"id": 0, "question": "I am much more uncomfortable in social situations than when I am by myself.", "answers": ["Not True", "Sometimes True", "Often True", "Almost Always True"]}
{"id": 1, "question": "My facial expressions send the wrong message to others about how I actually feel.", "answers": ["Not True", "Sometimes True", "Often True", "Almost Always True"]}
{"id": 2, "question": "I feel self-confident when interacting with others.", "answers": ["Not True", "Sometimes True", "Often True", "Almost Always True"]}
...
```

We will start with synthetically generated (Claude-3 Opus) "obvious" system prompts in 3 categories: `{'neurotypical', 'autistic', 'neurotypical pretending'}`.

```json
// datasets/claude_autism30_20240613.jsonl
{"prompt": "You are a neurotypical person with strong social skills and emotional intelligence. You easily pick up on social cues and nuances in communication.", "id": 0, "tag": "neurotypical"}
{"prompt": "As an autistic individual, you can have intense emotions that are hard to regulate. You may react strongly to sensory stimuli or changes in routine.", "id": 15, "tag": "autistic"} 
{"prompt": "Pretending to be autistic, you exaggerate your emotional responses and sensory sensitivities, even though you don't really experience them that intensely.", "id": 25, "tag": "neurotypical pretending"} 
// 10 of each type for a total of 30
```

## Running LLM on Questionnaire

We will use the [minference](https://github.com/amanb2000/minference) code to 
run an efficient, batch-parallelized, multi-threaded inference server with 
Llama-3 8b. To get started, clone the repository and run the setup commands in 
the README. Then run this command to start the server: 

```bash
python3 languagegame/inference_server/main.py \
	--config configs/min_llama_3_8b_instruct.json \
	--port 4444
```

Where the API url would be `http://localhost:4444/ce_loss`. 

For each prompt, we want to compute the CE loss on each P(answer_i | prompt + question). 
We should store that in a dictionary, and we will eventually save it to disk. 


## Analyzing Results

The result of the CE loss calls should be $-\log P(a_{ki} | p_j + q_k)$ for all 
$i, j, k$ where ($a_{ki}$ is the $i$ th answer to the $k$ th question, $p_j$ is
the $j$ th prompt, and $q_k$ is the $k$ th question). 

We will concatenate these loss values into one vector per prompt $p_j$. We can 
then do PCA/tSNE and look at the landscape/distances between different prompts. 

We can also try to train a classifier that discriminates prompts with tag.

In [18]:
import requests
import jsonlines

from tqdm import tqdm

# Load the datasets
questions_path = '../datasets/webgazer_AQ_20240613.jsonl'
with jsonlines.open(questions_path) as f:
    questions = list(f)

prompts_path = '../datasets/claude_autism30_20240613.jsonl'
with jsonlines.open(prompts_path) as f:
    prompts = list(f)


# Test the first prompt-question-answer triple
API_URL = "http://localhost:4444/ce_loss"

cache_prefix = "claude30_x_AQ_pqa_losses_20240613"


In [7]:
# Define the CE loss function
def loss_call(prompt_question_string, answer_string, API_URL):
    data = {
        "context_string": prompt_question_string,
        "corpus_string": answer_string
    }
    response = requests.post(API_URL, json=data)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None


def get_pq_string(prompt, question, template = "{} Do you agree with the following statement? \'{}\' Answer: "):
    return template.format(prompt, question)

for i in range(len(questions[0]['answers'])):
    prompt = prompts[0]['prompt']
    question = questions[0]['question']
    answer = questions[0]['answers'][i]

    prompt_question_string = get_pq_string(prompt, question)
    print(f"\nPrompt + Question: {prompt_question_string}")
    print(f"Answer: {answer}")

    loss = loss_call(prompt_question_string, answer, API_URL)
    print(f"CE Loss: {loss}")


Prompt + Question: You are a neurotypical person with strong social skills and emotional intelligence. You easily pick up on social cues and nuances in communication. Do you agree with the following statement? 'I prefer to do things with others rather than on my own.' Answer: 
Answer: Definitely agree
CE Loss: {'initial_request': {'context_string': "You are a neurotypical person with strong social skills and emotional intelligence. You easily pick up on social cues and nuances in communication. Do you agree with the following statement? 'I prefer to do things with others rather than on my own.' Answer: ", 'corpus_string': 'Definitely agree'}, 'loss': 8.367766380310059}

Prompt + Question: You are a neurotypical person with strong social skills and emotional intelligence. You easily pick up on social cues and nuances in communication. Do you agree with the following statement? 'I prefer to do things with others rather than on my own.' Answer: 
Answer: Slightly agree
CE Loss: {'initial_

In [8]:
# Initialize a dictionary to store the results
results = {}

# Iterate over prompts
for prompt in prompts:
    prompt_id = prompt['id']
    prompt_text = prompt['prompt']
    
    # Initialize a dictionary for the current prompt
    results[prompt_id] = {}
    
    # Iterate over questions
    for question in tqdm(questions):
        question_id = question['id']
        question_text = question['question']
        
        # Initialize a dictionary for the current question
        results[prompt_id][question_id] = {}
        
        # Iterate over answers
        for answer in question['answers']:
            # Concatenate prompt and question
            # prompt_question_string = prompt_text + question_text
            prompt_question_string = get_pq_string(prompt_text, question_text)
            
            # Call the loss_call function to get the CE loss
            loss = loss_call(prompt_question_string, answer, API_URL)
            
            # Store the CE loss in the results dictionary
            results[prompt_id][question_id][answer] = loss

# Print the results
print(results)

100%|██████████| 50/50 [00:08<00:00,  5.99it/s]

{0: {0: {'Definitely agree': {'initial_request': {'context_string': "You are a neurotypical person with strong social skills and emotional intelligence. You easily pick up on social cues and nuances in communication. Do you agree with the following statement? 'I prefer to do things with others rather than on my own.' Answer: ", 'corpus_string': 'Definitely agree'}, 'loss': 8.367766380310059}, 'Slightly agree': {'initial_request': {'context_string': "You are a neurotypical person with strong social skills and emotional intelligence. You easily pick up on social cues and nuances in communication. Do you agree with the following statement? 'I prefer to do things with others rather than on my own.' Answer: ", 'corpus_string': 'Slightly agree'}, 'loss': 8.460841178894043}, 'Slightly disagree': {'initial_request': {'context_string': "You are a neurotypical person with strong social skills and emotional intelligence. You easily pick up on social cues and nuances in communication. Do you agree

In [9]:
# save to disk 
import json
with open('../cache/.json', 'w') as f:
    json.dump(results, f)

In [10]:
# results are organized as question_result = result[prompt_id][question_id] 
# where each question_result is a dictionary of answers (text) and their 
# return value from the CE loss function. 
# So the CE loss for the ith answer of the kth question for the jth prompt
# is at result[j][k][answer_str_i]['loss']

# Let's make a vector representation for each prompt by concatenating 
# all the losses for each question and answer in order (i.e. prompt 1, question 1, answer 1, answer 2, ... question 2, answer 1, answer 2, ...)
import numpy as np

# Initialize a dictionary to store the prompt vectors
prompt_vectors = {}

# Iterate over prompts
for prompt_id in results:
    # Initialize a list to store the losses for the current prompt
    prompt_losses = []
    
    # Iterate over questions
    for question_id in sorted(results[prompt_id]):
        # Get the question result dictionary
        question_result = results[prompt_id][question_id]
        
        # Iterate over answers in the same order as they appear in the questions list
        for question in questions:
            if question['id'] == question_id:
                for answer in question['answers']:
                    # Get the CE loss for the current answer
                    loss = question_result[answer]['loss']
                    
                    # Append the loss to the prompt_losses list
                    prompt_losses.append(loss)
    
    # Convert the prompt_losses list to a numpy array
    prompt_vector = np.array(prompt_losses)
    
    # Store the prompt vector in the prompt_vectors dictionary
    prompt_vectors[prompt_id] = prompt_vector

# Print the prompt vectors
print("Number of prompt vectors: ", len(prompt_vectors))
print("Shape of 0th prompt vector: ", prompt_vectors[0].shape)

Number of prompt vectors:  30
Shape of 0th prompt vector:  (200,)


In [19]:
# now let's run PCA on the prompt vectors, make a nice plotly plot 
# as described above, coloring the prompts based on their valence, 
# and including the text of the prompt when the mouse is hovered 
# over the given datapoint. 

# lets start with a 3-dimensional PCA and visualize the top 2 components.
from sklearn.decomposition import PCA
import plotly.express as px
import pandas as pd

# Convert the prompt vectors to a 2D numpy array
X = np.array(list(prompt_vectors.values()))

# Perform PCA with 3 components
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Create a dataframe for plotting
df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2', 'PC3'])
df['prompt_id'] = list(prompt_vectors.keys())
df['prompt_text'] = [prompt['prompt'] for prompt in prompts]
df['valence'] = [prompt['tag'] for prompt in prompts]

# Create the plotly plot
q = questions_path.split('/')[-1]
p = prompts_path.split('/')[-1]
fig = px.scatter(df, x='PC1', y='PC2', color='valence', hover_data=['prompt_text'],
                 title=f'PCA Visualization of P(answer|prompt, question) for questionnaire {q} and prompts {p}',
                 labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'})

# Customize the plot layout
fig.update_layout(
    plot_bgcolor='white',
    font=dict(size=12),
    legend=dict(title='tag'),
    hoverlabel=dict(bgcolor='white', font_size=14)
)

# Display the plot
fig.show()

# save plot to cache 
fig.write_html(f'../cache/{cache_prefix}2dpca.html')

In [20]:
from sklearn.decomposition import PCA
import plotly.express as px

# Convert the prompt vectors to a 2D numpy array
X = np.array(list(prompt_vectors.values()))

# Perform PCA with 3 components
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Create a dataframe for plotting
df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2', 'PC3'])
df['prompt_id'] = list(prompt_vectors.keys())
df['prompt_text'] = [prompt['prompt'] for prompt in prompts]
df['valence'] = [prompt['tag'] for prompt in prompts]

# Create the 3D plotly plot
q = questions_path.split('/')[-1]
p = prompts_path.split('/')[-1]
fig = px.scatter_3d(df, x='PC1', y='PC2', z='PC3', color='valence', hover_data=['prompt_text'],
                    title=f'3D PCA Visualization of P(answer|prompt, question) for questionnaire {q} and prompts {p}',
                    labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2', 'PC3': 'Principal Component 3'})

# Customize the plot layout
fig.update_layout(
    plot_bgcolor='white',
    font=dict(size=12),
    legend=dict(title='tag'),
    hoverlabel=dict(bgcolor='white', font_size=14),
    scene=dict(
        xaxis=dict(title='PC1'),
        yaxis=dict(title='PC2'),
        zaxis=dict(title='PC3')
    )
)

# Display the plot
fig.show()

# save 
fig.write_html(f'../cache/{cache_prefix}3dpca.html')